COMMAND ----------

In [1]:

!pip install pdfservices-sdk
!pip install openpyxl
!pip install langchain==0.0.278
!pip install langchain-core==0.0.11
!pip install pdfservices-sdk
!pip install openai==0.27.8


Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/85.6 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.1 MB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/40.8 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/155.3 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/63.1 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from adobe.pdfservices.operation.auth.credentials import Credentials
from adobe.pdfservices.operation.exception.exceptions import ServiceApiException, ServiceUsageException, SdkException
from adobe.pdfservices.operation.execution_context import ExecutionContext
from adobe.pdfservices.operation.io.file_ref import FileRef
from adobe.pdfservices.operation.pdfops.extract_pdf_operation import ExtractPDFOperation
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_pdf_options import ExtractPDFOptions
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_element_type import ExtractElementType
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_renditions_element_type import \
    ExtractRenditionsElementType
import os.path
import zipfile
import json
import pandas as pd
import re
import openpyxl
from datetime import datetime

In [2]:
def get_dict_xlsx(outputzipextract, xlsx_file):
    """
    Function to read excel output from adobe API
    """
    # Read excel
    df = pd.read_excel(os.path.join(
        outputzipextract, xlsx_file), sheet_name='Sheet1', engine='openpyxl')
    
    # Clean df
    df.columns = [re.sub(r"_x([0-9a-fA-F]{4})_", "", col) for col in df.columns]
    df = df.replace({r"_x([0-9a-fA-F]{4})_": ""}, regex=True)

    # Convert df to string
    data_dict = df.to_dict(orient='records')

    return data_dict

In [4]:

client_id = os.getenv("ADOBE_CLIENT_ID")
client_secret = os.getenv('ADOBE_CLIENT_SECRET')
client_id = 'cc9a3889ee33401c98fc8e22b31a43e6'
client_secret = 'p8e-gUADUlOvUimilFf3NzypTzT9aJPNEsP2'

In [4]:
def adobeLoader(input_pdf, output_zip_path,client_id, client_secret):
    """
    Function to run adobe API and create output zip file
    """
    # Initial setup, create credentials instance.
    credentials = Credentials.service_principal_credentials_builder() \
        .with_client_id(client_id) \
        .with_client_secret(client_secret) \
        .build()

    # Create an ExecutionContext using credentials and create a new operation instance.
    execution_context = ExecutionContext.create(credentials)
    extract_pdf_operation = ExtractPDFOperation.create_new()

    # Set operation input from a source file.
    source = FileRef.create_from_local_file(input_pdf)
    extract_pdf_operation.set_input(source)

    # Build ExtractPDF options and set them into the operation
    extract_pdf_options: ExtractPDFOptions = ExtractPDFOptions.builder() \
        .with_elements_to_extract([ExtractElementType.TEXT, ExtractElementType.TABLES]) \
        .with_elements_to_extract_renditions([ExtractRenditionsElementType.TABLES,
                                                ExtractRenditionsElementType.FIGURES]) \
        .build()
    extract_pdf_operation.set_options(extract_pdf_options)

    # Execute the operation.
    result: FileRef = extract_pdf_operation.execute(execution_context)

    # Save result to output path
    if os.path.exists(output_zip_path):
        os.remove(output_zip_path)
    result.save_as(output_zip_path)

In [5]:
def extract_text_from_file_adobe(output_zip_path, output_zipextract_folder):
    """
    Function to extract text and table from adobe output zip file
    Read and Process JSON: Opens the structuredData.json file which contains the extracted data in a structured format.
    Data Extraction: Iterates through the elements in the JSON file, collecting text
    directly and processing tables by referring to their file paths in the Excel
    format. Table Processing via get_dict_xlsx: When it encounters a table, it uses
    the file path to call get_dict_xlsx, which reads the Excel file, cleans the
    data, and converts it to a dictionary format. This dictionary is then converted
    into a JSON string and added to the DataFrame.
    """    
    json_strings = []
    try:
        print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} unzip file")
        # Open the ZIP file
        with zipfile.ZipFile(output_zip_path, 'r') as zip_ref:
            # Extract all the contents of the ZIP file to the current working directory
            zip_ref.extractall(path=output_zipextract_folder)
    except Exception as e:
        print(f"Error processing input: {e}")

    try:
        print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} open json file")
        # Opening JSON file
        with open(os.path.join(output_zipextract_folder, "structuredData.json")) as json_file:
            data = json.load(json_file)
    except Exception as e:
        print(f"Error processing output: {e}")

    try:
        print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} extract text")
        dfs = pd.DataFrame()
        page = ''
        # Loop through elements in the document
        for ele in data['elements']:
            df = pd.DataFrame()

            # Get element page
            if ('Page' in ele.keys()):
                page = ele['Page']

            # Append table
            if any(x in ele['Path'] for x in ['Table']):
                if ('filePaths' in ele):
                    if [s for s in ele['filePaths'] if 'xlsx' in s]:
                        # Read excel table
                        data_dict = get_dict_xlsx(output_zipextract_folder, ele['filePaths'][0])
                        json_string = json.dumps(data_dict)
                        json_strings.append(json_string)
                        print (f"the json string is: {json_string}")
                        df = pd.DataFrame({'text': json_string}, index=[0])
                        

            # Append text            
            elif 'Text' in ele.keys():
                df = pd.DataFrame({'text': ele['Text']}, index=[0])

            # print(page)
            df['page_number'] = page
            dfs = pd.concat([dfs, df], axis=0)

        dfs = dfs.reset_index(drop=True)

        # Groupby page
        dfs = dfs.groupby('page_number')['text'].apply(lambda x: '\n'.join(x)).reset_index()
    except Exception as e:
        print('----Error: cannot extract text')
        return None, None
    return dfs, json_strings

In [8]:
# Adobe output zip file path
input_pdf = '../data/raw/GPTsareGPTs.pdf'
output_zip_path = '../data/processed/adobe_result/sdk.zip'
output_zipextract_folder = '../data/processed/adobe_result/'
# Run adobe API
adobeLoader(input_pdf, output_zip_path, client_id= client_id, client_secret=client_secret)
# unzip adobe output zipfile, extract text and table from adobe output zip file
df, json_strings = extract_text_from_file_adobe(output_zip_path, output_zipextract_folder)

2024-04-28 14:31:17 unzip file
2024-04-28 14:31:17 open json file
2024-04-28 14:31:17 extract text
the json string is: [{"Task ID ": "14675 ", "Occupation Title ": "Computer Systems Engineers/Architects ", "DWAs ": "Monitor computer system performance to ensure proper operation. ", "Task Description ": "Monitor system operation to detect potential problems. "}, {"Task ID ": "18310 ", "Occupation Title ": "Acute Care Nurses ", "DWAs ": "Operate diagnostic or therapeutic medical instruments or equipment. Prepare medical supplies or equipment for use. ", "Task Description ": "Set up, operate, or monitor invasive equipment and devices, such as colostomy or tracheotomy equipment, mechanical ventilators, catheters, gastrointestinal tubes, and central lines. "}, {"Task ID ": "4668.0 ", "Occupation Title ": "Gambling Cage Workers ", "DWAs ": "Execute sales or other financial transactions. ", "Task Description ": "Cash checks and process credit card advances for patrons. "}, {"Task ID ": "15709

In [13]:
print(df['text'].iloc[1])

Figure 1: Taken directly from GPT-4 Technical Report (OpenAI, 2023b). To get a sense of how quickly model capabilities are progressing – consider the jump in exam performance between GPT-3.5 and GPT-4 (OpenAI, 2023b). 
Our study is motivated less by the progress of these models alone though, and more by the breadth, scale, and capabilities we’ve seen in the complementary technologies developed around them. The role of complementary technologies remains to be seen, but maximizing the impact of LLMs appears contingent on integrating them with larger systems (Bresnahan, 2019; Agrawal et al., 2021). While the focus of our discussion is primarily on the generative capabilities of LLMs, it is important to note that these models can also be utilized for various tasks beyond text generation. For example, embeddings from LLMs can be used for custom search applications, and LLMs can perform tasks such as summarization and classification where the context may be largely contained in the prompt. 


In [14]:
df.head()

,page_number,text
0,0,GPTs are GPTs: An Early Look at the Labor Mark...
1,1,Figure 1: Taken directly from GPT-4 Technical ...
2,2,levels in GPT-4 responses and between human an...
3,3,policymakers to predict and regulate. As with ...
4,4,"Furthermore, a positive feedback loop may emer..."
